In [20]:
import os
import pandas as pd
from azure.identity.aio import ManagedIdentityCredential
cred = ManagedIdentityCredential()
opts = {'account_name': 'datalakestorage7nrw9r', 'credential': cred, 'anon': False}
folder_path = "abfs://datalakegen2filesystem/analytics/connectedcar/chenha01/Churn/"

In [21]:
train = pd.read_csv(folder_path + 'trainOneMonth_stratify.csv', delimiter=',', storage_options=opts)
test = pd.read_csv(folder_path + 'testOneMonth_stratify.csv', delimiter=',', storage_options=opts)
valid = pd.read_csv(folder_path + 'validOneMonth_stratify.csv', delimiter=',', storage_options=opts)

In [22]:
OneMonthbins = pd.read_csv(folder_path + 'OneMonthbinsMixOldNew.csv', delimiter=',', storage_options=opts)
# convert OneMonthbins into dictionary, with variable as key, the vraiable's dataframe as value
columns = OneMonthbins['variable'].unique()
OneMonthbins_dict = {}
for col in columns:
    OneMonthbins_dict[col] = OneMonthbins[OneMonthbins['variable'] == col]
# apply OneMonthbins
import scorecardpy as sc
train_woe = sc.woebin_ply(train, OneMonthbins_dict)
train_woe['OneMonth_target'] = train['OneMonth_target']
valid_woe = sc.woebin_ply(valid, OneMonthbins_dict)
valid_woe['OneMonth_target'] = valid['OneMonth_target']
test_woe = sc.woebin_ply(test, OneMonthbins_dict)
test_woe['OneMonth_target'] = test['OneMonth_target']

[INFO] converting into woe values ...
Woe transformating on 60880 rows and 1641 columns in 00:03:51
[INFO] converting into woe values ...
Woe transformating on 26092 rows and 1641 columns in 00:01:42
[INFO] converting into woe values ...
Woe transformating on 15846 rows and 1641 columns in 00:01:02


In [23]:
# get value of woe when variable is 'statecode' and bin is missing
colum = 'statecode'
missing_woe = OneMonthbins[OneMonthbins['variable'] == colum]['woe'].iloc[0]
# fill missing values in 'statecode' with missing_woe
colum = colum + '_woe'
train_woe[colum] = train_woe[colum].fillna(missing_woe)
valid_woe[colum] = valid_woe[colum].fillna(missing_woe)
test_woe[colum] = test_woe[colum].fillna(missing_woe)

In [15]:
from sklearn import linear_model

In [24]:
base_features = ['statecode_woe', 'age_drv_woe', 'appfeatures_woe', 'base_price_woe', 'months_paying_woe',
'countallvehiclesgasaddedpast6mths_woe', 'curraddrmedianincome_woe']
model = linear_model.LogisticRegression()
model.fit(train_woe[base_features], train_woe['OneMonth_target'])

LogisticRegression()

In [25]:
import numpy as np
from scipy import stats
import pandas as pd

# p value
# remove rows wuth missing values
valid_woe.dropna(inplace=True)
params = np.append(model.intercept_, model.coef_)
predictions = model.predict(valid_woe[base_features])
newX = np.append(np.ones((len(valid_woe), 1)), valid_woe[base_features], axis=1)
MSE = (sum((valid_woe['OneMonth_target'] - predictions) ** 2)) / (len(newX) - len(newX[0]))

# Compute the dot product
matrix = np.dot(newX.T, newX)

# Check if the matrix is singular
if np.linalg.det(matrix) != 0:
    inv_matrix = np.linalg.inv(matrix)
else:
    inv_matrix = np.linalg.pinv(matrix)

var_b = MSE * inv_matrix.diagonal()
sd_b = np.sqrt(var_b)
ts_b = params / sd_b
p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(newX) - 1))) for i in ts_b]

# summary
summary = pd.DataFrame()
summary["features"] = ['intercept'] + base_features
summary["coefficients"] = params
summary["p_values"] = p_values
summary

,features,coefficients,p_values
0,intercept,-0.984508,0.0
1,statecode_woe,0.906720,0.0
2,age_drv_woe,0.792169,0.0
3,appfeatures_woe,1.426953,0.0
4,base_price_woe,0.606935,0.0
5,months_paying_woe,0.840730,0.0
6,countallvehiclesgasaddedpast6mths_woe,0.453150,0.0
7,curraddrmedianincome_woe,0.333364,0.0


In [27]:
# return probability of each row in test
prob = model.predict_proba(valid_woe[base_features])
# return probability of 1
prob = [p[1] for p in prob]
# sort prob in order
valid_woe['prob'] = prob
valid_woe.sort_values(by='prob', ascending=True, inplace=True)
# group rows into 10 groups with same number of rows
valid_woe['group'] = pd.qcut(valid_woe['prob'], 10, labels=False)
valid_woe['OneMonth_target_Cnt'] = valid_woe['OneMonth_target']

# group by group, calculate the sum of OneMonth_target and count of OneMonth_target
test_group = valid_woe.groupby('group').agg({'OneMonth_target': 'sum', 'OneMonth_target_Cnt': 'count'}).reset_index()
# calculate rate of OneMonth_target by OneMonth_target /  pred
test_group['rate'] = test_group['OneMonth_target'] / test_group['OneMonth_target_Cnt']
# get rate of OneMonth_target from group as 0
rate_0_ = test_group[test_group['group'] == 0]['rate'].values[0]
rate_9_ = test_group[test_group['group'] == 9]['rate'].values[0]

# return probability of each row in test
prob = model.predict_proba(test_woe[base_features])
# return probability of 1
prob = [p[1] for p in prob]
# sort prob in order
test_woe['prob'] = prob
test_woe.sort_values(by='prob', ascending=True, inplace=True)
# group rows into 10 groups with same number of rows
test_woe['group'] = pd.qcut(test_woe['prob'], 10, labels=False)
test_woe['OneMonth_target_Cnt'] = test_woe['OneMonth_target']

# group by group, calculate the sum of OneMonth_target and count of OneMonth_target
test_group = test_woe.groupby('group').agg({'OneMonth_target': 'sum', 'OneMonth_target_Cnt': 'count'}).reset_index()
# calculate rate of OneMonth_target by OneMonth_target /  pred
test_group['rate'] = test_group['OneMonth_target'] / test_group['OneMonth_target_Cnt']
# get rate of OneMonth_target from group as 0
rate_0 = test_group[test_group['group'] == 0]['rate'].values[0]
rate_9 = test_group[test_group['group'] == 9]['rate'].values[0]

print(base_features)
print('train lift:', rate_9_ / rate_0_) 
print('test lift:', rate_9 / rate_0) 

['statecode_woe', 'age_drv_woe', 'appfeatures_woe', 'base_price_woe', 'months_paying_woe', 'countallvehiclesgasaddedpast6mths_woe', 'curraddrmedianincome_woe']
train lift: 3.3134333269867686
test lift: 3.3903373369087593


<h1>2024 Only<h1>

In [28]:
# 2024 dataset only. All 3 files are from 2024
train_2024 = pd.read_csv(folder_path + 'beforewoe_train_stratify.csv', delimiter=',', storage_options=opts)
valid_2024 = pd.read_csv(folder_path + 'beforewoe_valid_stratify.csv', delimiter=',', storage_options=opts)
test_2024 = pd.read_csv(folder_path + 'beforewoe_test_stratify.csv', delimiter=',', storage_options=opts)

In [29]:
OneMonthbins = pd.read_csv(folder_path + 'OneMonthbins.csv', delimiter=',', storage_options=opts)
# convert OneMonthbins into dictionary, with variable as key, the vraiable's dataframe as value
columns = OneMonthbins['variable'].unique()
OneMonthbins_dict = {}
for col in columns:
    OneMonthbins_dict[col] = OneMonthbins[OneMonthbins['variable'] == col]
# apply OneMonthbins
import scorecardpy as sc
train_woe = sc.woebin_ply(train_2024, OneMonthbins_dict)
train_woe['OneMonth_target'] = train_2024['OneMonth_target']
valid_woe = sc.woebin_ply(valid_2024, OneMonthbins_dict)
valid_woe['OneMonth_target'] = valid_2024['OneMonth_target']
test_woe = sc.woebin_ply(test_2024, OneMonthbins_dict)
test_woe['OneMonth_target'] = test_2024['OneMonth_target']

[INFO] converting into woe values ...
Woe transformating on 25880 rows and 1657 columns in 00:01:43
[INFO] converting into woe values ...
Woe transformating on 11092 rows and 1657 columns in 00:00:42
[INFO] converting into woe values ...
Woe transformating on 15846 rows and 1657 columns in 00:01:04


In [30]:
# get value of woe when variable is 'statecode' and bin is missing
colum = 'statecode'
missing_woe = OneMonthbins[OneMonthbins['variable'] == colum]['woe'].iloc[0]
# fill missing values in 'statecode' with missing_woe
colum = colum + '_woe'
train_woe[colum] = train_woe[colum].fillna(missing_woe)
valid_woe[colum] = valid_woe[colum].fillna(missing_woe)
test_woe[colum] = test_woe[colum].fillna(missing_woe)

In [31]:
base_features = ['statecode_woe', 'age_drv_woe', 'appfeatures_woe', 'base_price_woe', 'months_paying_woe',
'countallvehiclesgasaddedpast6mths_woe', 'curraddrmedianincome_woe']
model = linear_model.LogisticRegression()
model.fit(train_woe[base_features], train_woe['OneMonth_target'])

LogisticRegression()

In [32]:
import numpy as np
from scipy import stats
import pandas as pd

# p value
# remove rows wuth missing values
valid_woe.dropna(inplace=True)
params = np.append(model.intercept_, model.coef_)
predictions = model.predict(valid_woe[base_features])
newX = np.append(np.ones((len(valid_woe), 1)), valid_woe[base_features], axis=1)
MSE = (sum((valid_woe['OneMonth_target'] - predictions) ** 2)) / (len(newX) - len(newX[0]))

# Compute the dot product
matrix = np.dot(newX.T, newX)

# Check if the matrix is singular
if np.linalg.det(matrix) != 0:
    inv_matrix = np.linalg.inv(matrix)
else:
    inv_matrix = np.linalg.pinv(matrix)

var_b = MSE * inv_matrix.diagonal()
sd_b = np.sqrt(var_b)
ts_b = params / sd_b
p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(newX) - 1))) for i in ts_b]

# summary
summary = pd.DataFrame()
summary["features"] = ['intercept'] + base_features
summary["coefficients"] = params
summary["p_values"] = p_values
summary

,features,coefficients,p_values
0,intercept,-0.971276,0.000000
1,statecode_woe,0.939164,0.000000
2,age_drv_woe,0.768753,0.000000
3,appfeatures_woe,1.491288,0.000000
4,base_price_woe,0.517469,0.000000
5,months_paying_woe,0.884693,0.000000
6,countallvehiclesgasaddedpast6mths_woe,0.458657,0.000000
7,curraddrmedianincome_woe,0.245786,0.001663


In [33]:
# return probability of each row in test
prob = model.predict_proba(valid_woe[base_features])
# return probability of 1
prob = [p[1] for p in prob]
# sort prob in order
valid_woe['prob'] = prob
valid_woe.sort_values(by='prob', ascending=True, inplace=True)
# group rows into 10 groups with same number of rows
valid_woe['group'] = pd.qcut(valid_woe['prob'], 10, labels=False)
valid_woe['OneMonth_target_Cnt'] = valid_woe['OneMonth_target']

# group by group, calculate the sum of OneMonth_target and count of OneMonth_target
test_group = valid_woe.groupby('group').agg({'OneMonth_target': 'sum', 'OneMonth_target_Cnt': 'count'}).reset_index()
# calculate rate of OneMonth_target by OneMonth_target /  pred
test_group['rate'] = test_group['OneMonth_target'] / test_group['OneMonth_target_Cnt']
# get rate of OneMonth_target from group as 0
rate_0_ = test_group[test_group['group'] == 0]['rate'].values[0]
rate_9_ = test_group[test_group['group'] == 9]['rate'].values[0]

# return probability of each row in test
prob = model.predict_proba(test_woe[base_features])
# return probability of 1
prob = [p[1] for p in prob]
# sort prob in order
test_woe['prob'] = prob
test_woe.sort_values(by='prob', ascending=True, inplace=True)
# group rows into 10 groups with same number of rows
test_woe['group'] = pd.qcut(test_woe['prob'], 10, labels=False)
test_woe['OneMonth_target_Cnt'] = test_woe['OneMonth_target']

# group by group, calculate the sum of OneMonth_target and count of OneMonth_target
test_group = test_woe.groupby('group').agg({'OneMonth_target': 'sum', 'OneMonth_target_Cnt': 'count'}).reset_index()
# calculate rate of OneMonth_target by OneMonth_target /  pred
test_group['rate'] = test_group['OneMonth_target'] / test_group['OneMonth_target_Cnt']
# get rate of OneMonth_target from group as 0
rate_0 = test_group[test_group['group'] == 0]['rate'].values[0]
rate_9 = test_group[test_group['group'] == 9]['rate'].values[0]

print(base_features)
print('train lift:', rate_9_ / rate_0_) 
print('test lift:', rate_9 / rate_0) 

/tmp/ipykernel_18979/411852560.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  valid_woe['prob'] = prob
/tmp/ipykernel_18979/411852560.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  valid_woe['group'] = pd.qcut(valid_woe['prob'], 10, labels=False)
/tmp/ipykernel_18979/411852560.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented

['statecode_woe', 'age_drv_woe', 'appfeatures_woe', 'base_price_woe', 'months_paying_woe', 'countallvehiclesgasaddedpast6mths_woe', 'curraddrmedianincome_woe']
train lift: 2.8408291603586817
test lift: 3.603683329347046


/tmp/ipykernel_18979/411852560.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_woe['group'] = pd.qcut(test_woe['prob'], 10, labels=False)
/tmp/ipykernel_18979/411852560.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_woe['OneMonth_target_Cnt'] = test_woe['OneMonth_target']
